# Get General Ledger Data
Source: https://data.ok.gov/dataset/general-ledger

## Imports

In [ ]:
from __future__ import annotations
from typing import List, Tuple
import os
from pathlib import Path
import subprocess
import requests
from rich import print
import pandas as pd
import duckdb

# file encoding (detected mixed encoding in the hosted files)
from charset_normalizer import detect

## Constants

In [ ]:
# Oklahoma Government dataset
DOMAIN = "https://data.ok.gov"
ACTION = "/api/3/action/package_show"
URL = f"{DOMAIN}/{ACTION}"
PARAMS = {
    "id": "general-ledger"
}

# Local storage
DATA_DIR = Path("../data/gl")
DATA_RAW = DATA_DIR / "01_raw"
DATA_SILVER = DATA_DIR / "02_silver"
DATA_GOLD = DATA_DIR / "03_gold"
LOCAL_DB = Path("../data/local_db")
LOCAL_DATA = [DATA_DIR, DATA_RAW, DATA_SILVER, DATA_GOLD, LOCAL_DB]

for p in LOCAL_DATA:
    p.mkdir(parents=True, exist_ok=True)

# Local DB connection
con = duckdb.connect(f"{str(LOCAL_DB)}/ledger.duckdb")
con

## Available Resources
- Datasets available from FY22 to FY26 Q3
  - FY26 most likely to be incomplete
- Extract the filename and download urls

In [4]:
# Retrieve filenames and download urls
response = requests.get(url=URL, params=PARAMS)
response.raise_for_status()

data = response.json()\
    .get('result', [])\
    .get('resources', [])

# filename and download url
url_collections = [
    (f['name'], f['url']) for f in data
    if f['url'].lower().endswith('.csv')
]

print(url_collections)

[
    (
        'LEDGER_FY22_QTR1.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/6fe771bd-a705-4fce-b12f-d0a041ac
5581/download/ledger_fy22_qtr1.csv'
    ),
    (
        'LEDGER_FY22_QTR2.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/a1434f0b-9f3d-4726-b5a3-cef6ce61
f7c9/download/ledger_fy22_qtr2.csv'
    ),
    (
        'LEDGER_FY22_QTR3.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/c3e91ddc-70a4-4da5-ac39-50ab7a63
ed01/download/ledger_fy22_qtr3.csv'
    ),
    (
        'LEDGER_FY22_QTR4.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/9bbced1c-285b-4060-baa6-93d77159
d03e/download/ledger_fy22_qtr4.csv'
    ),
    (
        'LEDGER_FY23_QTR2.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/dc0f53a9-4b1e-4848-87b9-2f06cd6f
c627/download/ledger_fy23_qtr2.csv'
    ),
    (
        'LEDGER_FY23_QTR3.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/8d8a2b2f-7141-4b9f-8661-8ddba9d9
a29a/download/ledger_fy23_qtr3.csv'
    ),
    (
        'LEDGER_FY23_QTR4.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/553fa5c5-3143-4c54-83a2-cf36d02b
7e67/download/ledger_fy23_qtr4.csv'
    ),
    (
        'LEDGER_FY24_QTR2.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/90e09488-4a3d-4b33-b8e6-85c6358f
28ac/download/ledger_fy24_qtr2.csv'
    ),
    (
        'LEDGER_FY24_QTR3.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/5f09e519-28d7-4d1a-bf99-c55966b7
8b19/download/ledger_fy24_qtr3.csv'
    ),
    (
        'LEDGER_FY24_QTR4.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/0c2a7e42-610e-4f1d-826f-f6b267fb
a68a/download/ledger_fy24_qtr4.csv'
    ),
    (
        'LEDGER_FY25_QTR2.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/b1b031e1-d26d-4f1f-8892-4fcec010
31dc/download/ledger_fy25_qtr2.csv'
    ),
    (
        'LEDGER_FY23_QTR1.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/85c77eec-842b-4a3c-b6dd-176c9b29
6a0f/download/ledger_fy23_qtr1.csv'
    ),
    (
        'LEDGER_FY24_QTR1.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/2aa3deaa-859d-46ee-baa4-c2ef94a8
3ac5/download/ledger_fy24_qtr1.csv'
    ),
    (
        'LEDGER_FY25_QTR1.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/a8fa19ca-8265-41bf-a1f9-59c8993d
72f8/download/ledger_fy25_qtr1.csv'
    ),
    (
        'LEDGER_FY25_QTR3.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/67e2656c-acfe-431d-8667-ad8b2418
0ce5/download/ledger_fy25_qtr3.csv'
    ),
    (
        'LEDGER_FY25_QTR4.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/72038a7c-ad10-4d67-82cd-4f269207
27e1/download/ledger_fy25_qtr4.csv'
    ),
    (
        'LEDGER_FY26_QTR1.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/b8f6518e-9cd9-4bf5-b21e-743c9aea
bf98/download/ledger_fy26_qtr1.csv'
    ),
    (
        'LEDGER_FY26_QTR2.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/fab28aae-e33c-49bf-8b42-bb1dfbf8
3eee/download/ledger_fy26_qtr2.csv'
    ),
    (
        'LEDGER_FY26_QTR3.csv',
        'https://data.ok.gov/dataset/dd1ecf41-4abc-4886-ab0f-b84d7662d8d4/resource/b0578bcb-d8e8-45e9-b58d-f9e0d64e
7b5f/download/ledger_fy26_qtr3.csv'
    )
]

## Download Locally
- Store in `raw` and write to duckdb
- Use `curl` to download the files in bulk
- Files have varying encoding types (`ascii` and `windows-1250`)
  - Handle in cleaning phase

In [5]:
def download_files(url_collections: List[Tuple[str, str]]):
    # download the data and provide the filename
    for fname, url in url_collections:
        fname = str((DATA_RAW) / fname)
        subprocess.run(
            args=['curl', '-L', '-o', fname, url],
            check=True
        )
        yield fname


# Raw local db data
table_created = False

# Download files / append to duckdb
for file in download_files(url_collections):
    try:
        # Files appear to be ascii encoded
        df = pd.read_csv(file, encoding="ascii")
    except UnicodeDecodeError:
        try:
            print(f"Retrying {file=} with a different encoding")
            with open(file, 'rb') as f:
                encoding = detect(f.read())['encoding']
                df = pd.read_csv(file, encoding=encoding)
        except Exception as e:
            print(f"Failed to process {file}: {e}")
            continue
    
    # Create the initial table
    if not table_created:
        con.execute("CREATE TABLE ledger_raw AS SELECT * FROM df LIMIT 0")
        table_created = True

    # Append to the table
    try:
        con.execute("INSERT INTO ledger_raw SELECT * FROM df")
    except Exception as e:
        print(f"Failed to append {file} to DuckDB: {e}")
    
    print(f"Saved and appended {file=}")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1085    0  1085    0     0   4166      0 --:--:-- --:--:-- --:--:--  4173
100  121M  100  121M    0     0  13.2M      0  0:00:09  0:00:09 --:--:-- 14.0M


Saved and appended file='../data/gl/01_raw/LEDGER_FY22_QTR1.csv'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1085    0  1085    0     0   4881      0 --:--:-- --:--:-- --:--:--  4887
100 98.9M  100 98.9M    0     0  13.1M      0  0:00:07  0:00:07 --:--:-- 15.2M


Saved and appended file='../data/gl/01_raw/LEDGER_FY22_QTR2.csv'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1085    0  1085    0     0   4246      0 --:--:-- --:--:-- --:--:--  4254
100  160M  100  160M    0     0  15.8M      0  0:00:10  0:00:10 --:--:-- 17.5M


...

Saved and appended file='../data/gl/01_raw/LEDGER_FY26_QTR1.csv'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1085    0  1085    0     0   5093      0 --:--:-- --:--:-- --:--:--  5117
100  140M  100  140M    0     0  11.3M      0  0:00:12  0:00:12 --:--:-- 12.8M
/var/folders/2w/3xfrnv6j5315lxwkwg55grqh0000gn/T/ipykernel_32651/4271069354.py:19: DtypeWarning: Columns (0: CLASS_FLD) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding="ascii")


Saved and appended file='../data/gl/01_raw/LEDGER_FY26_QTR2.csv'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1085    0  1085    0     0   5117      0 --:--:-- --:--:-- --:--:--  5142
100 73.3M  100 73.3M    0     0  13.4M      0  0:00:05  0:00:05 --:--:-- 14.6M
/var/folders/2w/3xfrnv6j5315lxwkwg55grqh0000gn/T/ipykernel_32651/4271069354.py:19: DtypeWarning: Columns (0: CLASS_FLD) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, encoding="ascii")


Saved and appended file='../data/gl/01_raw/LEDGER_FY26_QTR3.csv'